In [45]:
import torch
import config
from utils import (
    get_model_instance_segmentation,
    collate_fn,
    get_transform,
    myOwnDataset,
)

print("Torch version:", torch.__version__)


Torch version: 2.1.2+cu121


In [46]:
# create own Dataset
my_dataset = myOwnDataset(
    root=config.train_data_dir, annotation=config.train_coco, transforms=get_transform()
)

# own DataLoader
data_loader = torch.utils.data.DataLoader(
    my_dataset,
    batch_size=config.train_batch_size,
    shuffle=config.train_shuffle_dl,
    num_workers=config.num_workers_dl,
    collate_fn=collate_fn,
)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [47]:
# select device (whether GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    # print(annotations)


model = get_model_instance_segmentation(config.num_classes)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params, lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay
)

len_dataloader = len(data_loader)

[{'boxes': tensor([[227.0000, 492.0000, 289.5000, 559.0000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([16], device='cuda:0'), 'area': tensor([4187.5000], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[205.0000, 238.0000, 279.0000, 385.5000],
        [339.0000, 250.0000, 407.0000, 380.5000]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id': tensor([4], device='cuda:0'), 'area': tensor([10915.,  8874.], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}, {'boxes': tensor([[129.0000, 414.0000, 295.5000, 578.5000]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([27], device='cuda:0'), 'area': tensor([27389.2500], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}, {'boxes': tensor([[141.,  64., 294., 211.]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([3], device='cuda:0'), 'area': tensor([22491.], device='cuda:0'), 

In [48]:
# Training
for epoch in range(config.num_epochs):
    print(f"Epoch: {epoch}/{config.num_epochs}")
    model.train()
    i = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        # Debug
        # print(annotations)
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(f"Iteration: {i}/{len(data_loader)}, Loss: {losses}")

Epoch: 0/2
Iteration: 1/12, Loss: 1.3066667318344116
Iteration: 2/12, Loss: 1.1441655158996582
Iteration: 3/12, Loss: 0.9766808748245239
Iteration: 4/12, Loss: 0.9287682175636292
Iteration: 5/12, Loss: 0.8460279703140259
Iteration: 6/12, Loss: 0.7232286930084229
Iteration: 7/12, Loss: 0.5605407357215881
Iteration: 8/12, Loss: 0.46249663829803467
Iteration: 9/12, Loss: 0.5101029872894287
Iteration: 10/12, Loss: 0.31631985306739807
Iteration: 11/12, Loss: 0.4155082702636719
Iteration: 12/12, Loss: 0.2142888754606247
Epoch: 1/2
Iteration: 1/12, Loss: 0.3003377914428711
Iteration: 2/12, Loss: 0.25758159160614014
Iteration: 3/12, Loss: 0.3800334930419922
Iteration: 4/12, Loss: 0.35076072812080383
Iteration: 5/12, Loss: 0.21907278895378113
Iteration: 6/12, Loss: 0.18857194483280182
Iteration: 7/12, Loss: 0.27473384141921997
Iteration: 8/12, Loss: 0.15770868957042694
Iteration: 9/12, Loss: 0.17649364471435547
Iteration: 10/12, Loss: 0.2872791588306427
Iteration: 11/12, Loss: 0.267810791730880